In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import genjax
genjax.pretty()

In [ ]:
import jax
from jax.random import key as prngkey
import jax.numpy as jnp
from condorgmm.condor.pose import Pose
import condorgmm.condor.model.model as model
from condorgmm.condor.config import DEFAULT_HYPERPARAMS
from condorgmm.condor.types import CondorGMMState, MyPytree, SingleKnownObjectSceneState
import condorgmm
from condorgmm.condor.rerun import log_state

# init_model_simulate = (model.init_model.simulate)



In [ ]:
# Generate an object model
hypers = DEFAULT_HYPERPARAMS.replace(
    n_gaussians=10,
    datapoint_mask=jnp.ones(1, dtype=bool),
    running_simulate=jnp.array(True, dtype=bool),
)
tr = model.init_model.simulate(prngkey(1), (hypers,))
gaussians = tr.get_retval().gaussians

In [ ]:
condorgmm.rr_init("objmodel_00")
log_state(tr.get_retval(), hypers)

In [ ]:
# Generate a condorgmmScene with this as the object model
scene = SingleKnownObjectSceneState(
    transform_World_Camera=Pose.identity(),
    transform_World_Object=Pose(jnp.array([500, 500, 500.]), jnp.array([0., 0., 0., 1.])),
    background_rigidity = jnp.array(0.003, dtype=jnp.float32),
    object_model=gaussians
)
hypers = DEFAULT_HYPERPARAMS.replace(
    n_gaussians=100,
    initial_scene=scene,
    datapoint_mask=jnp.ones(1000, dtype=bool),
    running_simulate=jnp.array(True, dtype=bool),
)
tr = model.init_model.simulate(prngkey(0), (hypers,))


In [ ]:
from condorgmm.condor.model.model import get_object_model_gaussian_in_camera_frame, gaussian_is_background
get_object_model_gaussian_in_camera_frame(91, scene, hypers)
gaussian_is_background(89, hypers)

In [ ]:
tr.get_retval().gaussians[85:95]

In [ ]:
scene.transform_World_Camera.inv().pos

In [ ]:
import rerun as rr
condorgmm.rr_init("objmodel_01")
rr.set_time_sequence("frame", 0)
log_state(tr.get_retval(), hypers)

In [ ]:
step_jit = jax.jit(model.step_model.simulate)

In [ ]:
hyp2 = hypers.replace(
    object_pose_drift_std=jnp.array(100, dtype=float)
)

st = tr.get_retval()
for i in range(100):
    rr.set_time_sequence("frame", i)
    st = step_jit(prngkey(i), (st, hyp2)).get_retval()
    log_state(st, hyp2)


In [ ]:
rr.set_time_sequence("frame", 1)
log_state(tr2.get_retval(), hypers)